In [23]:
from torchvision.models import vgg16_bn
import torch.nn as nn
import torch

In [2]:
net = vgg16_bn(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to C:\Users\shug4421/.cache\torch\hub\checkpoints\vgg16_bn-6c64b313.pth
100.0%


In [9]:
numFtrs = net.classifier[6].in_features
net.classifier[6] = nn.Linear(numFtrs,12)

In [19]:
net.features[0]

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [20]:
numChannels = net.features[0].out_channels
net.features[0] = nn.Conv2d(7, numChannels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [24]:
testImg = torch.randn((1,7,288,384))
out = net(testImg)

In [25]:
out

tensor([[ 0.4267, -0.2684, -0.1402, -0.0163,  0.2919,  0.1560,  0.6093,  0.3154,
          0.0391, -0.0258, -0.4329,  0.3354]], grad_fn=<AddmmBackward>)